# Knowledge Bases for Amazon Bedrock - Comparing Vector Store Options

This notebook demonstrates creating Amazon Bedrock Knowledge Bases with two different vector store backends:
1. **Amazon OpenSearch Serverless (AOSS)** - Managed vector search with millisecond latency
2. **Amazon S3 Vectors** - Cost-effective vector storage with sub-second latency (Preview)

Both Knowledge Bases will use the same data source (Amazon S3 bucket with shareholder letters), allowing you to compare their performance and characteristics in the next notebook.

#### Notebook Walkthrough

We will create two parallel data pipelines that ingest the same documents into two different Knowledge Bases:

**AOSS Knowledge Base:**
- Traditional vector database approach
- Millisecond query latency
- Higher cost for large datasets
- Mature, production-ready

**S3 Vectors Knowledge Base:**
- Native S3 vector storage (Preview)
- Sub-second query latency
- Cost-effective for large datasets
- Seamless S3 integration

![data_ingestion.png](./images/data_ingestion.png)

### High-Level Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     Amazon S3 Data Source                        │
│                  (Shareholder Letters PDFs)                      │
└────────────────────────┬────────────────────────────────────────┘
                         │
                         │ Same documents ingested into both KBs
                         │
         ┌───────────────┴───────────────┐
         │                               │
         ▼                               ▼
┌─────────────────────┐         ┌─────────────────────┐
│   AOSS Knowledge    │         │ S3 Vectors Knowledge│
│       Base          │         │       Base          │
│                     │         │                     │
│  ┌───────────────┐  │         │  ┌───────────────┐  │
│  │ Titan Embed   │  │         │  │ Titan Embed   │  │
│  │  Text v2      │  │         │  │  Text v2      │  │
│  │ (1024 dim)    │  │         │  │ (1024 dim)    │  │
│  └───────┬───────┘  │         │  └───────┬───────┘  │
│          │          │         │          │          │
│          ▼          │         │          ▼          │
│  ┌───────────────┐  │         │  ┌───────────────┐  │
│  │  OpenSearch   │  │         │  │  S3 Vectors   │  │
│  │  Serverless   │  │         │  │    Index      │  │
│  │  Collection   │  │         │  │               │  │
│  │               │  │         │  │  - Cosine     │  │
│  │  - HNSW       │  │         │  │  - Float32    │  │
│  │  - FAISS      │  │         │  │  - 1024 dim   │  │
│  │  - L2 distance│  │         │  │               │  │
│  └───────────────┘  │         │  └───────────────┘  │
└─────────────────────┘         └─────────────────────┘
         │                               │
         │                               │
         └───────────────┬───────────────┘
                         │
                         ▼
              ┌─────────────────────┐
              │  Bedrock Agent      │
              │  Runtime API        │
              │                     │
              │  - Retrieve         │
              │  - RetrieveAndGen   │
              └─────────────────────┘
                         │
                         ▼
              ┌─────────────────────┐
              │   Nova/Claude/etc   │
              │   (Generation)      │
              └─────────────────────┘
```

#### Steps: 
1. Create IAM execution roles with policies for both vector stores
2. Create Amazon OpenSearch Serverless collection and index
3. Create S3 Vectors bucket and index
4. Download sample documents (Amazon shareholder letters)
5. Create S3 bucket as data source and upload documents
6. Create two Knowledge Bases (one for each vector store)
7. Create data sources connecting both KBs to the same S3 bucket
8. Start ingestion jobs for both Knowledge Bases

Once both Knowledge Bases are ready, you can compare their performance in:
- [02_managed-rag-kb-retrieve-generate-api.ipynb](02_managed-rag-kb-retrieve-generate-api.ipynb)


In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Imports and shared configuration

In [ ]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role, create_aoss_policy_attach_bedrock_execution_role, create_policies_in_aoss, interactive_sleep_for
import random
from retrying import retry
import time

import sys
sys.path.append('../')
from util.tagging import standard_tags, standard_tags_kv, standard_tags_kv_lc
from util.model_selector import create_text_model_selector

In [ ]:
region_name = os.environ.get("AWS_REGION", "us-east-1")
#Clients
bedrock_agent_client = boto3.client(service_name="bedrock-agent", region_name=region_name)
sts_client = boto3.client(service_name="sts", region_name=region_name)
s3_client = boto3.client(service_name="s3", region_name=region_name)
aoss_client = boto3.client(service_name="opensearchserverless", region_name=region_name)
s3vectors_client = boto3.client('s3vectors', region_name=region_name)
iam_client = boto3.client('iam')

#Extra params
suffix = random.randrange(200, 900)
timestamp = str(int(time.time()))
account_id = sts_client.get_caller_identity()["Account"]

#Buckets and vector store names
s3_suffix = f"{region_name}-{account_id}"
source_bucket_name = f'bedrock-kb-{s3_suffix}-{suffix}'
vector_store_name_aoss = f'bedrock-sample-rag-aoss-{suffix}'
index_name_aoss = f"bedrock-sample-rag-index-aoss-{suffix}"
vector_bucket_name = f"bedrock-kb-s3vectors-{suffix}-{timestamp}"
vector_index_name = f"bedrock-kb-index-{suffix}"
pp = pprint.PrettyPrinter(indent=2)

#Knowledge base config for aoss
aoss_kb_name = f"bedrock-sample-knowledge-base-aoss-{suffix}"
aoss_kb_description = "Amazon shareholder letter Knowledge Base backed by AOSS."


# Knowledge Base configuration for S3 Vectors
s3vectors_kb_name = f"bedrock-s3vectors-knowledge-base-{suffix}"
s3vectors_kb_description = "Amazon shareholder letter Knowledge Base with S3 Vectors storage."


# Ingest strategy - How to ingest data from the data source
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# The data source to ingest documents from, into the OpenSearch serverless and S3 Vector Knowledge Base index
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{source_bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a Knowledge Base using data within S3 prefixes.
}

# The embedding model used by Bedrock to embed ingested documents, and realtime prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v2:0"

#Must match the embedding dimension of the selected embedding model
s3_vector_dimension = 1024  # Titan Embed Text v2 dimension

### Create S3 bucket data source for Knowledge Bases

In [ ]:
# Check if bucket exists, and if not create S3 bucket for Knowledge Base data source
try:
    s3_client.head_bucket(Bucket=source_bucket_name)
    print(f'Bucket {source_bucket_name} Exists')
except ClientError as e:
    print(f'Creating bucket {source_bucket_name}')
    if region_name == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=source_bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
        Bucket=source_bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region_name }
    )
    s3_client.put_bucket_tagging(
        Bucket=source_bucket_name,
        Tagging={
            'TagSet': standard_tags_kv
        }
    )

In [ ]:
%store source_bucket_name

### Download data to ingest into our S3 Data Source

In [ ]:
# Download and prepare dataset
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)


#### Upload data to S3 Bucket data source

In [ ]:
# Upload data to s3 to the bucket that was configured as a data source to the Knowledge Base
def uploadDirectory(path,bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucket_name,file)

uploadDirectory(data_root, source_bucket_name)

## Create a vector store - Amazon OpenSearch Serverless (AOSS) index

### Step 1 - Create AOSS policies and collection
First of all we have to create a vector store. In this section we will use *Amazon OpenSearch Serverless.*

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [ ]:
bedrock_kb_execution_role_aoss = create_bedrock_execution_role(bucket_name=source_bucket_name)
bedrock_kb_execution_role_arn_aoss = bedrock_kb_execution_role_aoss['Role']['Arn']

Create an Amazon OpenSeach Serverless collection for the vector store. Note that creation of the collection can take several minutes. You can use the Amazon OpenSearch Serverless console to monitor creation progress.

In [ ]:
# create security, network and data access policies within AOSS
encryption_policy, network_policy, access_policy = create_policies_in_aoss(
    vector_store_name=vector_store_name_aoss,
    aoss_client=aoss_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn_aoss)
collection = aoss_client.create_collection(name=vector_store_name_aoss,type='VECTORSEARCH', tags=standard_tags_kv_lc)

In [ ]:
pp.pprint(collection)

In [ ]:
%store encryption_policy network_policy access_policy collection

In [ ]:
# Get the OpenSearch serverless collection URL
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

In [ ]:
# wait for collection creation
# This can take couple of minutes to finish
def collection_created():    
    response = aoss_client.batch_get_collection(names=[vector_store_name_aoss])
    return response['collectionDetails'][0]['status'] != 'CREATING'

interactive_sleep_for(collection_created)

In [ ]:
# create opensearch serverless access policy and attach it to Bedrock execution role
try:
    create_aoss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                    bedrock_kb_execution_role=bedrock_kb_execution_role_aoss)
except Exception as e:
    print("Policy already exists")
    pp.pprint(e)

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, AuthorizationException, RequestError, AuthenticationException
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, region_name, 'aoss')

# Build the OpenSearch client
oss_api_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

## Step 2 - Create vector index

We will create the vector index in Amazon Opensearch Serverless, with the `knn_vector` type, specifying the dimension size, name, and engine.
Read the [OpenSearch documentation on k-NN vector](https://docs.opensearch.org/latest/field-types/supported-field-types/knn-vector/) for more details.


In [ ]:
body_json = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1024,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"
         }
      }
   }
}

In [ ]:
# we need to retry as it can take a minute to propagate the security policies to AOSS
@retry(retry_on_exception=lambda e: isinstance(e, (AuthenticationException, AuthorizationException)),
       wait_fixed=5000,
       stop_max_delay=60*1000)
def create_index():
    # Create index
    try:
        response = oss_api_client.indices.create(index=index_name_aoss, body=json.dumps(body_json))
        print('\nCreating index:')
        pp.pprint(response)
    except RequestError as e:
        if e.error == 'resource_already_exists_exception':
            # oss_api_client.indices.delete(index=index_name_aoss)
            print("Index already exists. You can delete the index if its already exists by the delete line in this cell.")
        else:
            raise

create_index()

## Create Knowledge Base
Steps:
- Initialize Open search serverless configuration which will include collection ARN, index name, vector field, text field and metadata field.
- Initialize chunking strategy, based on which Knowledge Base will split the documents into pieces of size equal to the chunk size mentioned in the `chunkingStrategyConfiguration`.
- Initialize the s3 configuration, which will be used to create the data source object later.
- Initialize the Titan Embed Text model ARN, as this will be used to create the embeddings for each of the text chunks.

In [ ]:
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name_aoss,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

Provide the above configurations as input to the `create_knowledge_base` method, which will create the Knowledge Base.

In [ ]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_aoss_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = aoss_kb_name,
        description = aoss_kb_description,
        roleArn = bedrock_kb_execution_role_arn_aoss,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        },
        tags=standard_tags
    )
    return create_kb_response["knowledgeBase"]

In [ ]:
try:
    kb_aoss = create_knowledge_base_aoss_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

pp.pprint(kb_aoss)   

In [ ]:
# Get KnowledgeBase 
get_kb_response_aoss = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_aoss['knowledgeBaseId'])

## Create S3 Vectors Knowledge Base

### Step 1 - Create S3 Vectors bucket and index

Amazon S3 Vectors (Preview) provides cost-effective vector storage integrated directly with S3. It offers:
- **Cost savings** for large vector datasets
- **Sub-second query latency** for retrieval operations  
- **Seamless S3 integration** with familiar AWS services
- **Automatic scaling** without infrastructure management

Let's create an S3 Vectors bucket and index for our second Knowledge Base.


In [ ]:
print(f"S3 Vectors Configuration:")
print(f"  Bucket: {vector_bucket_name}")
print(f"  Index: {vector_index_name}")
print(f"  Dimension: {s3_vector_dimension}")

# Create S3 Vectors bucket
s3vectors_client.create_vector_bucket(
    vectorBucketName=vector_bucket_name,
    encryptionConfiguration={'sseType': 'AES256'},
)
print(f"✅ Created S3 Vectors bucket: {vector_bucket_name}")

# Create vector index with cosine distance metric
s3vectors_client.create_index(
    vectorBucketName=vector_bucket_name,
    indexName=vector_index_name,
    dataType="float32",
    dimension=s3_vector_dimension,
    distanceMetric="cosine",
    metadataConfiguration={
            "nonFilterableMetadataKeys": [
                "AMAZON_BEDROCK_TEXT"
            ]
        }
)
print(f"✅ Created vector index: {vector_index_name}")

# Get the index ARN for Knowledge Base configuration
vector_index_arn = f"arn:aws:s3vectors:{region_name}:{account_id}:bucket/{vector_bucket_name}/index/{vector_index_name}"
print(f"Vector Index ARN: {vector_index_arn}")

### Step 2 - Create IAM execution role for S3 Vectors Knowledge Base

The execution role needs additional permissions for S3 Vectors operations compared to AOSS.

In [ ]:
# Create IAM policy for S3 Vectors access
s3vectors_policy_name = f'AmazonBedrockS3VectorsPolicyForKnowledgeBase_{suffix}'

s3vectors_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockInvokeModelPermission",
            "Effect": "Allow",
            "Action": ["bedrock:InvokeModel"],
            "Resource": [embeddingModelArn]
        },
        {
            "Sid": "S3ListBucketPermission",
            "Effect": "Allow",
            "Action": ["s3:ListBucket"],
            "Resource": [f"arn:aws:s3:::{source_bucket_name}"],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": [account_id]
                }
            }
        },
        {
            "Sid": "S3GetObjectPermission",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [f"arn:aws:s3:::{source_bucket_name}/*"],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": [account_id]
                }
            }
        },
        {
            "Sid": "S3VectorsAccessPermission",
            "Effect": "Allow",
            "Action": [
                "s3vectors:GetIndex",
                "s3vectors:QueryVectors",
                "s3vectors:PutVectors",
                "s3vectors:GetVectors",
                "s3vectors:DeleteVectors"
            ],
            "Resource": vector_index_arn,
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": account_id
                }
            }
        }
    ]
}

# Create the policy
s3vectors_policy = iam_client.create_policy(
    PolicyName=s3vectors_policy_name,
    PolicyDocument=json.dumps(s3vectors_policy_document),
    Description='Policy for S3 Vectors access from Bedrock Knowledge Base',
    Tags=standard_tags_kv
)

s3vectors_policy_arn = s3vectors_policy["Policy"]["Arn"]
print(f"Created S3 Vectors policy: {s3vectors_policy_arn}")

# Create execution role for S3 Vectors Knowledge Base
s3vectors_role_name = f'AmazonBedrockExecutionRoleForS3Vectors_{suffix}'

assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "bedrock.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

s3vectors_kb_execution_role = iam_client.create_role(
    RoleName=s3vectors_role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy),
    Description='Amazon Bedrock Knowledge Base Execution Role for S3 Vectors',
    MaxSessionDuration=3600,
    Tags=standard_tags_kv
)

s3vectors_kb_execution_role_arn = s3vectors_kb_execution_role['Role']['Arn']

# Attach the S3 Vectors policy
iam_client.attach_role_policy(
    RoleName=s3vectors_role_name,
    PolicyArn=s3vectors_policy_arn
)

print(f"Created S3 Vectors execution role: {s3vectors_kb_execution_role_arn}")

# Wait for role to propagate
print("Waiting for IAM role to propagate...")
time.sleep(10)

### Step 3 - Create S3 Vectors Knowledge Base

Now we'll create the second Knowledge Base using S3 Vectors as the storage backend.

In [ ]:
# S3 Vectors storage configuration
s3VectorsConfiguration = {
    "indexArn": vector_index_arn
}

print(f"Creating S3 Vectors Knowledge Base: {s3vectors_kb_name}")

# Create S3 Vectors Knowledge Base
@retry(wait_random_min=1000, wait_random_max=2000, stop_max_attempt_number=7)
def create_s3vectors_knowledge_base():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name=s3vectors_kb_name,
        description=s3vectors_kb_description,
        roleArn=s3vectors_kb_execution_role_arn,
        knowledgeBaseConfiguration={
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn,
                "embeddingModelConfiguration": {
                    "bedrockEmbeddingModelConfiguration": {
                        "dimensions": s3_vector_dimension,
                        "embeddingDataType": "FLOAT32"
                    }
                }
            }
        },
        storageConfiguration={
            "type": "S3_VECTORS",
            "s3VectorsConfiguration": s3VectorsConfiguration
        },
        tags=standard_tags
    )
    return create_kb_response["knowledgeBase"]

try:
    s3vectors_kb = create_s3vectors_knowledge_base()
    pp.pprint(s3vectors_kb)
except Exception as err:
    print(f"Error creating S3 Vectors Knowledge Base: {err=}, {type(err)=}")

## Create data sources for Knowledge Bases and start sync jobs

### Step 1 - Create data sources for both Knowledge Bases

Now we'll create data sources for both Knowledge Bases, connecting them to the same S3 bucket with our documents.


In [ ]:
# Create a DataSource for AOSS Knowledge Base 
create_ds_response = bedrock_agent_client.create_data_source(
    name = aoss_kb_name,
    description = aoss_kb_description,
    knowledgeBaseId = kb_aoss['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)

ds_aoss = create_ds_response["dataSource"]
pp.pprint(ds_aoss)

# Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb_aoss['knowledgeBaseId'], dataSourceId = ds_aoss["dataSourceId"])

In [ ]:
# Create data source for S3 Vectors Knowledge Base
create_s3vectors_ds_response = bedrock_agent_client.create_data_source(
    name=s3vectors_kb_name,
    description=s3vectors_kb_description,
    knowledgeBaseId=s3vectors_kb['knowledgeBaseId'],
    dataSourceConfiguration={
        "type": "S3",
        "s3Configuration": s3Configuration
    },
    vectorIngestionConfiguration={
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)

s3vectors_ds = create_s3vectors_ds_response["dataSource"]
pp.pprint(s3vectors_ds)

# Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = s3vectors_kb['knowledgeBaseId'], dataSourceId = s3vectors_ds["dataSourceId"])

### Step 2 - Start ingestion jobs for both Knowledge Bases

We'll start ingestion jobs for both Knowledge Bases simultaneously. This will process the same documents and create embeddings in both vector stores.


### Start ingestion job
Once the Knowledge Base and data source is created, we can start the ingestion job.
During the ingestion job, Knowledge Base will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case AOSS.

In [ ]:
# we need to retry as it can take a minute so the previous settings are effective
@retry(retry_on_exception=lambda e: isinstance(e, (bedrock_agent_client.exceptions.ValidationException, bedrock_agent_client.exceptions.ConflictException)),
       wait_fixed=5000,
       stop_max_delay=60*1000)
def start_job():
    return bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb_aoss['knowledgeBaseId'], dataSourceId = ds_aoss["dataSourceId"])

start_job_response = start_job()

In [ ]:
job_aoss = start_job_response["ingestionJob"]
pp.pprint(job_aoss)

In [ ]:
# Get job 
def job_completed():
  global job_aoss
  get_job_response = bedrock_agent_client.get_ingestion_job(
    knowledgeBaseId = kb_aoss['knowledgeBaseId'],
    dataSourceId = ds_aoss["dataSourceId"],
    ingestionJobId = job_aoss["ingestionJobId"]
  )
  job_aoss = get_job_response["ingestionJob"]
  return job_aoss['status']=='COMPLETE'

interactive_sleep_for(job_completed)

In [ ]:
# Start ingestion job for S3 Vectors Knowledge Base
@retry(retry_on_exception=lambda e: isinstance(e, (bedrock_agent_client.exceptions.ValidationException, bedrock_agent_client.exceptions.ConflictException)),
       wait_fixed=5000,
       stop_max_delay=60*1000)
def start_s3vectors_job():
    return bedrock_agent_client.start_ingestion_job(
        knowledgeBaseId=s3vectors_kb['knowledgeBaseId'],
        dataSourceId=s3vectors_ds["dataSourceId"]
    )

start_s3vectors_job_response = start_s3vectors_job()
s3vectors_job = start_s3vectors_job_response["ingestionJob"]

print("Started S3 Vectors ingestion job:")
pp.pprint(s3vectors_job)

# Wait for S3 Vectors ingestion job to complete
def s3vectors_job_completed():
    global s3vectors_job
    get_job_response = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=s3vectors_kb['knowledgeBaseId'],
        dataSourceId=s3vectors_ds["dataSourceId"],
        ingestionJobId=s3vectors_job["ingestionJobId"]
    )
    s3vectors_job = get_job_response["ingestionJob"]
    return s3vectors_job['status'] == 'COMPLETE'

print("Waiting for S3 Vectors ingestion job to complete...")
interactive_sleep_for(s3vectors_job_completed)
print("✅ S3 Vectors ingestion complete!")

In [ ]:
# Print the Knowledge Base Id in bedrock, that corresponds to the Opensearch index in the collection we created before, we will use it for the invocation later
kb_id_aoss = kb_aoss["knowledgeBaseId"]
kb_id_s3vectors = s3vectors_kb["knowledgeBaseId"]

print(f"\n📊 Knowledge Base Summary:")
print(f"  AOSS KB ID: {kb_id_aoss}")
print(f"  S3 Vectors KB ID: {kb_id_s3vectors}")

In [ ]:
# keep the kb_id for invocation later in the invoke request
%store kb_id_aoss
%store kb_id_s3vectors

### Additional Chllenges
- Read the [blog post](https://aws.amazon.com/blogs/machine-learning/amazon-bedrock-knowledge-bases-now-supports-metadata-filtering-to-improve-retrieval-accuracy) on metadata filtering
- Go to knowledgebases_and_rag/data directory and review the metadata files
- Add "year" attributte to each of the metadata files
- Upload the files to S3, and sync the knowledge base from the AWS console
- Test the knowledge base from the knowledge bast test environment in the AWS console, use Filters section with format key = value

## Summary

✅ Successfully created two Knowledge Bases:

1. **AOSS Knowledge Base** (`kb_id_aoss`)
   - Vector Store: Amazon OpenSearch Serverless
   - Latency: Milliseconds
   - Best for: Production workloads requiring ultra-low latency

2. **S3 Vectors Knowledge Base** (`kb_id_s3vectors`)
   - Vector Store: Amazon S3 Vectors (Preview)
   - Latency: Sub-second
   - Best for: Cost-effective large-scale vector storage

Both Knowledge Bases:
- Use the same embedding model (Titan Embed Text v2)
- Have the same chunking strategy (512 tokens, 20% overlap)
- Are connected to the same S3 data source
- Contain the same Amazon shareholder letters

You can now proceed to the next notebook to compare their retrieval performance:
- [02_managed-rag-kb-retrieve-generate-api.ipynb](02_managed-rag-kb-retrieve-generate-api.ipynb)
